# Unit 6: Queries against CoW tables

In this unit, we will learn about query types against CoW tables.<br>

We will review various types of queries/operations:<br>
Commit listing: This is a metadata listing of commits against a table<br>
Incremental queries: Queries only see new data written to the table, since a given commit/compaction. This effectively provides change streams to enable incremental data pipelines.<br>
Time travel queries: Queries can show the state of a record over time specified<br>
Point in time queries: Queries for data at a specific point in time<br>


This unit takes about 5 minutes to complete.

### Initialize Spark Session

In [1]:
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql.types import LongType
import pyspark.sql.functions as F
from datetime import datetime

spark = SparkSession.builder \
  .appName("Hudi-Learning-Unit-06-PySpark") \
  .master("yarn") \
  .enableHiveSupport() \
  .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog") \
  .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension") \
  .getOrCreate()

spark

23/08/01 18:59:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Variables

In [2]:
PROJECT_ID_OUTPUT=!gcloud config get-value core/project
PROJECT_ID=PROJECT_ID_OUTPUT[0]
PROJECT_NBR_OUTPUT=!gcloud projects describe $PROJECT_ID --format="value(projectNumber)"
PROJECT_NBR=PROJECT_NBR_OUTPUT[0]

TRIP_DATE='2021-01-31'
LOCATION="us-central1"
HUDI_COW_BASE_GCS_URI = f"gs://gaia_data_bucket-{PROJECT_NBR}/nyc-taxi-trips-hudi-cow"
DATAPROC_METASTORE_THRIFT_URI_LIST=!gcloud metastore services list --location $LOCATION | grep thrift | cut -d' ' -f11
DATAPROC_METASTORE_THRIFT_URI=DATAPROC_METASTORE_THRIFT_URI_LIST[0]

print(f"Project ID is {PROJECT_ID}")
print(f"Project number is {PROJECT_NBR}")
print(f"Project location is {LOCATION}")
print(f"Hudi base Cow table GCS URI is {HUDI_COW_BASE_GCS_URI}")
print(f"Dataproc Metastore Service thrift URI is {DATAPROC_METASTORE_THRIFT_URI}")
print(f"Trip date to be used for deletes is {TRIP_DATE}")

Project ID is apache-hudi-lab
Project number is 623600433888
Project location is us-central1
Hudi base Cow table GCS URI is gs://gaia_data_bucket-623600433888/nyc-taxi-trips-hudi-cow
Dataproc Metastore Service thrift URI is thrift://10.60.192.28:9080
Trip date to be used for deletes is 2021-01-31


## 1. Listing the commits to a table

In [3]:
spark.sql("call show_commits(table => 'taxi_db.nyc_taxi_trips_hudi_cow', limit => 100);").show(100, truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/08/01 18:59:28 WARN GhfsStorageStatistics: Detected potential high latency for operation op_open. latencyMs=104; previousMaxLatencyMs=0; operationCount=1; context=gs://gaia_data_bucket-623600433888/nyc-taxi-trips-hudi-cow/.hoodie/hoodie.properties


+-----------------+-------------------+-----------------+-------------------+------------------------+---------------------+----------------------------+------------+
|commit_time      |total_bytes_written|total_files_added|total_files_updated|total_partitions_written|total_records_written|total_update_records_written|total_errors|
+-----------------+-------------------+-----------------+-------------------+------------------------+---------------------+----------------------------+------------+
|20230801033306138|1355990            |0                |1                  |1                       |32603                |0                           |0           |
|20230801033217250|1356037            |0                |1                  |1                       |32604                |1                           |0           |
|20230801032820936|8891097            |0                |2                  |2                       |244892               |1                           |0           

## 2. Incremental queries
Queries only see new data written to the table, since a given commit/compaction. This effectively provides change streams to enable incremental data pipelines

In [4]:
# Read the base data
spark. \
  read. \
  format("hudi"). \
  load(HUDI_COW_BASE_GCS_URI). \
  createOrReplaceTempView("hudi_cow_table")

23/08/01 18:59:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
# Query for commits
commits = list(map(lambda row: row[0], spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_cow_table order by commitTime").collect()))
beginTime = commits[len(commits) - 2] # commit time we are interested in

print(f"Begin commit time is: {beginTime}")

# The various commit timestamps are:
for commit in commits:
    print(commit)

Begin commit time is: 20230801032340309
20230731203923454
20230731205659112
20230731210155584
20230731210745246
20230801032340309
20230801032820936


In [6]:
# Incrementally query data
incremental_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.begin.instanttime': beginTime,
}

tripsIncrementalDF = spark.read.format("hudi"). \
  options(**incremental_read_options). \
  load(HUDI_COW_BASE_GCS_URI)
tripsIncrementalDF.createOrReplaceTempView("hudi_trips_incremental")

spark.sql("SELECT `_hoodie_commit_time`, * FROM  hudi_trips_incremental LIMIT 100").show()
# If you strictly followed the lab untis, notebook by notebook, the results should reflect the upsert notebook records 

+-------------------+-------------------+--------------------+------------------+----------------------+--------------------+---------+---------+----------+--------+---------+-----------+---------+-------------------+-------------------+-----------------+---------+------------------+-------------------+---------------+-------------+------------+-----------+-----------+-----------+------------+---------------------+------------+-----------------+--------------------+---------+---------+--------------+------------------------+--------------------+-------------+---------+
|_hoodie_commit_time|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|taxi_type|trip_year|trip_month|trip_day|trip_hour|trip_minute|vendor_id|    pickup_datetime|   dropoff_datetime|store_and_forward|rate_code|pickup_location_id|dropoff_location_id|passenger_count|trip_distance| fare_amount|  surcharge|    mta_tax| tip_amount|tolls_amount|improvement_surcharge|total_

## 3. Time Travel queries
The sample below shows the state of a trip from inception to latest commit - across commits.

### 3.1. Using Spark SQL against HMS
Replace the query below with the trip ID of the UPDATE_CANDIDATE_TRIP_ID from notebook 4

In [7]:
for commit in commits:
    print(f"As of commit timestamp: {commit}")
    spark.sql("select _hoodie_commit_time,trip_id,taxi_type,vendor_id,pickup_datetime," \
              "dropoff_datetime,total_amount "\
              "from taxi_db.nyc_taxi_trips_hudi_cow "\
              f"timestamp as of {commit} "\
              "Where trip_date='2019-01-15' AND trip_id=695784702201").show()

As of commit timestamp: 20230731203923454


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230731205659112


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230731210155584


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230731210745246


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230801032340309


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230801032820936


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230801032820936|695784702201|   yellow|        1|2019-01-15 11:00:30|2019-01-15 11:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+



### 3.2. Using Dataframe API

In [8]:
from pyspark.sql.functions import col

for commit in commits:
    print(f"As of commit timestamp: {commit}")
    spark.read. \
      format("hudi"). \
      option("as.of.instant", commit) \
        .load(f"{HUDI_COW_BASE_GCS_URI}/trip_date=2019-01-15") \
        .filter((col("trip_id")  == 695784702201)) \
        .select("_hoodie_commit_time","trip_id","taxi_type","vendor_id","pickup_datetime","dropoff_datetime","total_amount") \
        .show()
    


As of commit timestamp: 20230731203923454


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230731205659112


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230731210155584


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230731210745246


23/08/01 19:01:25 WARN GhfsStorageStatistics: Detected potential high latency for operation stream_read_operations. latencyMs=509; previousMaxLatencyMs=66; operationCount=3778; context=gs://gaia_data_bucket-623600433888/nyc-taxi-trips-hudi-cow/.hoodie/hoodie.properties


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230801032340309


23/08/01 19:01:32 WARN GhfsStorageStatistics: Detected potential high latency for operation stream_write_operations. latencyMs=413; previousMaxLatencyMs=11; operationCount=9410; context=gs://dataproc-temp-us-central1-623600433888-ojsvfynx/92fae947-ff2e-4e99-9649-afae6a74a071/spark-job-history/application_1690608046061_0120.inprogress


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230731203923454|695784702201|   yellow|        1|2019-01-15 10:00:30|2019-01-15 10:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+

As of commit timestamp: 20230801032820936


+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|_hoodie_commit_time|     trip_id|taxi_type|vendor_id|    pickup_datetime|   dropoff_datetime|total_amount|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+
|  20230801032820936|695784702201|   yellow|        1|2019-01-15 11:00:30|2019-01-15 11:49:46|36.000000000|
+-------------------+------------+---------+---------+-------------------+-------------------+------------+



## 4. Point in time queries
Can be used to see the state of a record in a commit time range

### 4.1. Using Dataframe API

In [ ]:
# pyspark
beginTime = "000" # Represents all commits > this time.
endTime = commits[len(commits) - 2]

# query point in time data
point_in_time_read_options = {
  'hoodie.datasource.query.type': 'incremental',
  'hoodie.datasource.read.end.instanttime': endTime,
  'hoodie.datasource.read.begin.instanttime': beginTime
}

tripsPointInTimeDF = spark.read.format("hudi"). \
  options(**point_in_time_read_options). \
  load(f"{HUDI_COW_BASE_GCS_URI}/trip_date=2019-01-15"). \
  filter((col("trip_id")  == 695784702201)). \
  select("_hoodie_commit_time","trip_id","taxi_type","vendor_id","pickup_datetime","dropoff_datetime","total_amount"). \
  show()

23/08/01 19:01:42 WARN GhfsStorageStatistics: Detected potential high latency for operation op_get_file_status. latencyMs=283; previousMaxLatencyMs=273; operationCount=212; context=gs://gaia_data_bucket-623600433888/nyc-taxi-trips-hudi-cow/trip_date=2021-07-28/15fd4fed-fbc0-483a-8998-f906c438ca43-0_81-57-15502_20230731210155584.parquet
23/08/01 19:01:42 WARN GhfsStorageStatistics: Detected potential high latency for operation op_get_file_status. latencyMs=330; previousMaxLatencyMs=283; operationCount=220; context=gs://gaia_data_bucket-623600433888/nyc-taxi-trips-hudi-cow/trip_date=2019-05-05/9650d10d-af6e-4106-adc3-8bca114be29d-0_407-19-9369_20230731203923454.parquet
23/08/01 19:01:42 WARN GhfsStorageStatistics: Detected potential high latency for operation op_get_file_status. latencyMs=352; previousMaxLatencyMs=330; operationCount=223; context=gs://gaia_data_bucket-623600433888/nyc-taxi-trips-hudi-cow/trip_date=2019-04-23/6c7b99c7-71ad-4e91-8776-618277296276-0_155-19-9117_202307312039

### 4.2. Using SparkSQL against Dataproc Metastore Service

In [ ]:
spark.sql("SELECT _hoodie_commit_time,trip_id,taxi_type,vendor_id,pickup_datetime," \
              "dropoff_datetime,total_amount "\
              "FROM taxi_db.nyc_taxi_trips_hudi_cow "\
              "WHERE trip_date='2019-01-15' AND trip_id=695784702201 " \
              "AND _hoodie_commit_time between 20230731203923454 and 20230801032820936").show()

This concludes the unit, please proceed to the next notebook.